# Classification using LLM  
Classifying text into labels  
1. Sentiment
2. Language
3. Style (formal, informal etc.)
4. Covered topics
5. Political tendency

## Overview
Tagging has a few components:
1. `function`: Like extraction, tagging uses functions to specify how the model should tag a document
2. `schema`: Define how we want to tag the document

In [17]:
# Imports
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from enum import Enum

In [2]:
TEMPERATURE = 0.0
NUM_PREDICT = 256
MODEL = "gemma3:12b-it-qat"

model = init_chat_model(
    model=MODEL,
    temperature=TEMPERATURE,
    num_predict=NUM_PREDICT,
    use_gpu=True,
    model_provider='ollama',
)

In [22]:
# Define the prompt template
tagging_prompt = ChatPromptTemplate.from_template(
    """
    Extract the desired information from the following passage.

    Only extract the properties mentioned in the 'CombinedOutput' section.
    
    Passage:
    {input}
    """
)

In [23]:

class Sentiment(str, Enum):
    """
    Enum for sentiment classification.
    """
    positive = "positive"
    negative = "negative"
    neutral = "neutral"

class Classification(BaseModel):
    """
    Classification model to extract properties from the passage.
    """
    sentiment: Sentiment = Field(desctription='The sentiment of the text')
    aggressiveness: int = Field(description='How aggressive the text is on a scale from 1 to 10')
    language: str = Field(description='The language of the text')
    topic: str = Field(description='The topic of the text')

class WordCount(BaseModel):
    """
    Word count model to extract the number of words in the passage.
    """
    word_count: int = Field(description='The number of words in the text')

class CombinedOutput(BaseModel):
    """
    Combined output model to hold both classification and word count.
    """
    classification: Classification = Field(description='The classification of the text')
    word_count: WordCount = Field(description='The word count of the text')


In [24]:
# LLM
llm = model.with_structured_output(
    CombinedOutput
)

In [25]:
inp = "Singapore is a strong financial hub in Asia. The economy is growing rapidly, and the government is supportive of the business environment"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

In [26]:
print(response)

classification=Classification(sentiment=<Sentiment.positive: 'positive'>, aggressiveness=0, language='english', topic='economy') word_count=WordCount(word_count=23)
